In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import pingouin as pg
from scipy.stats import shapiro, normaltest
import numpy as np
import kdephys as kde

import warnings
warnings.filterwarnings('ignore')

probe_ord = ['NNXr', 'NNXo']

from acr.plots import lrg
plt.rcdefaults()
lrg()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')
from acr.utils import *

In [ ]:
%connect_info

In [ ]:
scd = {}
#scd['ACR_29'] = [1, 2, 3, 4]
#scd['ACR_44'] = [14]
#scd['ACR_37'] = [4]

In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'control'
MAIN_COLOR = CTRL_BLUE

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/firing_rates'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=False, extra_rebounds=False)
#reb_hd = {}
#for subject, exp in zip(subjects, exps):
#    reb_hd[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True, extra_rebounds=True)

In [ ]:
fr_dfs = {}
for subject, exp in zip(subjects, exps):
    fr = dag.load_raw_fr_df(subject, exp, data_dir=f'{acr.utils.pub_data_root}/raw_fr/by_probe')
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject], )
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='condition', ref_to_val='early_sd', col_to_relativize='fr', on=['probe'])
    fr_dfs[subject] = fr
frdf_all = pl.concat(fr_dfs.values())

# STIMULATION FIRING RATE

In [ ]:
frdf = pu.drop_sub_channels(frdf_all, scd)

In [ ]:
stimfr = frdf.filter(pl.col('condition')=='stim')
#stimfr = pu.drop_sub_channels(stimfr, scd)
stimfr_means = stimfr.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
stimfr_means = stimfr_means.sort(['subject', 'probe'])
#stim_chan_means = stimfr.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=stim_chan_means, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_during_entire_stim'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = stimfr_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = stimfr_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
# stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# REBOUND FIRING RATE

In [ ]:
fr_dfs = {}
for subject, exp in zip(subjects, exps):
    fr = dag.load_raw_fr_df(subject, exp, data_dir=f'{acr.utils.pub_data_root}/raw_fr/by_probe')
    fr = acr.hypnogram_utils.label_df_with_hypno_conditions(fr, hyp_dicts[subject], )
    fr = acr.hypnogram_utils.label_df_with_states(fr, full_hyps[subject])
    fr = acr.hypnogram_utils.label_df_with_full_bl(fr, state='NREM', col='datetime')
    fr = dag.relativize_df(fr, ref_to_col='full_bl', ref_to_val='True', col_to_relativize='fr', on=['probe'])
    fr_dfs[subject] = fr
frdf_all = pl.concat(fr_dfs.values())

In [ ]:
frdf = pu.drop_sub_channels(frdf_all, scd)

In [ ]:
rebfr = frdf.filter(pl.col('condition')=='rebound')
rebfr_means = rebfr.group_by(['subject', 'probe']).agg(pl.col('fr_rel').mean())
rebfr_means = rebfr_means.sort(['subject', 'probe'])
#reb_chan_means = rebfr.group_by(['subject', 'probe', 'channel']).agg(pl.col('fr_rel').mean())
#g = sns.catplot(data=reb_chan_means, x='channel', y='fr_rel', hue='probe', palette=[NNXR_GRAY, MAIN_COLOR], hue_order=probe_ord, row='subject', height=6, aspect=4, kind='bar')

In [ ]:
fig_id = 'firing_rate_during_NREM_REBOUND_1HR'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

nnxr = rebfr_means.prb('NNXr')['fr_rel'].to_numpy()
nnxo = rebfr_means.prb('NNXo')['fr_rel'].to_numpy()

f, ax = acr.plots.gen_paired_boxplot(nnxr, nnxo, colors=[NNXR_GRAY, MAIN_COLOR])
ax.set_xticklabels(['Contra. Control', 'Optrode'])
f.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats